# Прогнозирование стоимости автомобиля по характеристикам

# Подготовительные операции

In [ ]:
!pip install -q automl-alex

In [ ]:
import numpy as np
import pandas as pd
import sys
import re

from sklearn.model_selection import KFold,train_test_split
from sklearn.tree import ExtraTreeRegressor, DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier, GradientBoostingRegressor, ExtraTreesRegressor, BaggingRegressor, VotingRegressor

from tqdm.notebook import tqdm

from catboost import CatBoostRegressor

import automl_alex
from automl_alex import AutoMLRegressor, BestSingleModelRegressor

from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer

from pandas_profiling import ProfileReport

pd.set_option('max_columns', None)

In [ ]:
def mape(y_true, y_pred):
    return round(np.mean(np.abs((y_pred - y_true) / y_true)*100), 3)

In [ ]:
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

In [ ]:
DIR_TRAIN  = '../input/autoru-parsed-0603-1304/'
DIR_TEST   = '../input/sf-dst-car-price/'
VAL_SIZE   = 0.25
N_FOLDS    = 5

# Подгружаем данные

In [ ]:
train = pd.read_csv(DIR_TRAIN+'new_data_99_06_03_13_04.csv')
test = pd.read_csv(DIR_TEST+'test.csv')
sample_submission = pd.read_csv(DIR_TEST+'sample_submission.csv')

# EDA

С помощью пакета Pandas Profiling легко осуществляем EDA анализ датасета и получаем все нужные нам характеристики в одном месте.

In [ ]:
#profile = ProfileReport(train, title="Pandas Profiling Report")

In [ ]:
#profile.to_notebook_iframe()

In [ ]:
#profile.to_file("your_report.html")

# Data Preprocessing

In [ ]:
train = train.loc[(train.brand == "BMW")]
train = train.drop_duplicates()

In [ ]:
train.info()

In [ ]:
train.sample(3)

In [ ]:
test.info()

In [ ]:
test.sample(3)

In [ ]:
# Для корректного объединения датасетов переименуем vehicleConfiguration в vehicleTransmission в train
train.rename(columns={'vehicleConfiguration': 'vehicleTransmission'}, inplace=True)

# Для корректной обработки признаков объединяем трейн и тест в один датасет
train['sample'] = 1 # помечаем где у нас трейн
test['sample'] = 0 # помечаем где у нас тест
test['Price'] = 0 # в тесте у нас нет значения price, мы его должны предсказать, поэтому пока просто заполняем нулями

data = test.append(train, sort=False).reset_index(drop=True) # объединяем

In [ ]:
data.sample(5)

In [ ]:
#
data.drop(['brand', 'name', 'vehicleConfiguration', 'engineDisplacement', 'description', 'Комплектация', 'Руль', 'Состояние', 'Таможня', 'id', 'Unnamed: 0'], axis=1, inplace=True,)

# Обработаем признак "Владение"
data['bodyType'] = data['bodyType'].apply(lambda x: str(x))
def bodyType(row):
    for body_type in ['внедорожник', 'хэтчбек', 'купе-хардтоп', 'седан', 'универсал', 
                      'родстер', 'кабриолет', 'компактвэн', 'лифтбек', 'купе']:
        if row.lower().startswith(body_type):
            return body_type
data['bodyType'] = data['bodyType'].apply(bodyType)
data['bodyType'] = data['bodyType'].apply(lambda x: 'купе' if x == 'купе-хардтоп' else x)

# Обработаем признак "color"
data['color'] = data['color'].map({'CACECB': 'серебристый', 'FAFBFB':'белый', 'EE1D19':'красный', '97948F':'серый', 
                                     '660099':'пурпурный', '040001':'чёрный','4A2197':'фиолетовый', 
                                     '200204':'коричневый','0000CC':'синий', '007F00':'зелёный', 'C49648':'бежевый',
                                     '22A0F8':'голубой','DEA522':'золотистый','FFD600': 'жёлтый', 'FF8649':'оранжевый',
                                     'FFC0CB':'розовый'})

# Обработаем признак "vehicleTransmission"
data['vehicleTransmission'] = data['vehicleTransmission'].map({'AUTOMATIC': 'автоматическая', 'MECHANICAL': 'механическая', 'ROBOT': 'роботизированная',})

# Обработаем признак "enginePower"
data['enginePower'] = data['enginePower'].apply(lambda x: x if isinstance(x, float) else int(str(x).split()[0]))

# Обработаем признак "Владельцы"
data['Владельцы'] = data['Владельцы'].apply(lambda x: x if isinstance(x, float) else int(str(x).split()[0]))

# Обработаем признак "ПТС"
data['ПТС'] = data['ПТС'].map({'ORIGINAL': 'Оригинал', 'DUPLICATE': 'Дубликат'})

# Mеняем названия колонок
data.columns = ['bodyType', 'color', 'fuelType', 'modelDate', 'numberOfDoors',
                'productionDate', 'vehicleTransmission', 'enginePower', 'mileage',
                'drive', 'numOwn', 'docs', 'durOwn', 'sample', 'Price']

# Удалаяем необработанные признаки
data.drop(['durOwn',], axis=1, inplace=True,)

In [ ]:
data.sample(5)

In [ ]:
data['numOwn'] = data['numOwn'].fillna(data['numOwn'].value_counts().idxmax())

for column in ['bodyType', 'color', 'fuelType', 'vehicleTransmission', 'drive', 'docs']:
    data[column] = data[column].astype('category').cat.codes  

#data = pd.get_dummies(data, columns=['bodyType', 'color', 'fuelType', 'vehicleTransmission', 'drive', 'docs'])

for column in ['modelDate', 'numberOfDoors', 'productionDate', 'enginePower', 'mileage', 'numOwn']:
        data[column] = data[column].astype('int32')

In [ ]:
profile = ProfileReport(data, title="Pandas Profiling Report")
profile.to_widgets()

In [ ]:
train_data = data.query('sample == 1').drop(['sample'], axis=1)
test_data = data.query('sample == 0').drop(['sample'], axis=1)
data = train_data

y = train_data['Price'].values
X = train_data.drop(['Price'], axis=1)
X_sub = test_data.drop(['Price'], axis=1)

# Train Split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=VAL_SIZE, shuffle=True, random_state=RANDOM_SEED)

# Fit&Submit AutoML

AutoMLRegressor

In [ ]:
model = AutoMLRegressor(X_train, y_train, X_test, random_state = RANDOM_SEED, verbose = 1)


In [ ]:
predict_test, predict_train = model.fit_predict(timeout = 8000, verbose = 1)

In [ ]:
model.stack_models_cfgs

In [ ]:
print('MAPE train:  ', mape(y_train, predict_train), '%')
print('MAPE test:  ', mape(y_test, predict_test), '%')

In [ ]:
#predict_submission = model.predict(X_sub)
#sample_submission['price'] = predict_submission
#sample_submission['price'] = sample_submission['price'].apply(lambda x: round(x/1000)*1000)
#sample_submission.to_csv(f'automl_ensemble.csv', index=False)

BestSingleModelRegressor

In [ ]:
model = BestSingleModelRegressor(X_train, y_train, X_test, random_state = RANDOM_SEED, verbose = 1)

In [ ]:
history = model.opt(timeout=8000, verbose=1)

In [ ]:
model.history_trials_dataframe.head(5)

In [ ]:
#model.plot_opt_history()

In [ ]:
predicts = model.predict()
predicts.head(5)

In [ ]:
print('MAPE train:  ', mape(y_train, predicts['predict_train'][0]), '%')
print('MAPE test:  ', mape(y_test, predicts['predict_test'][0]), '%')

In [ ]:
#predict_submission = model.predict(X_sub)
#sample_submission['price'] = predict_submission
#sample_submission['price'] = sample_submission['price'].apply(lambda x: round(x/1000)*1000)
#sample_submission.to_csv(f'automl_single.csv', index=False)

# Fit&Submit H2O

In [ ]:
#import h2o
#from h2o.automl import H2OAutoML
#h2o.init()

In [ ]:
#frame = h2o.H2OFrame(data)
#frame.describe()

In [ ]:
#y = "Price"

In [ ]:
#aml = H2OAutoML(max_runtime_secs = 600, seed = RANDOM_SEED, project_name = "rds_03")
#aml.train(y = y, training_frame = frame)

In [ ]:
#aml.leaderboard.head()

In [ ]:
#predicts = aml.predict(frame)
#predicts.head(5)

# Fit&Submit FastAI

# Fit&Submit LightGBM